# Distances and Angles between Images

We are going to compute distances and angles between images.

## Learning objectives
By the end of this notebook, you will learn to

1. Write programs to compute distance.
2. Write programs to compute angle.

"distance" and "angle" are useful beyond their usual interpretation. They are
useful for describing __similarity__ between objects. You will
first use the functions you wrote to compare MNIST digits. Furthermore, we will
use these concepts for implementing the K Nearest Neighbors algorithm, which is
a useful algorithm for classifying object according to distance.

In [ ]:
# PACKAGE: DO NOT EDIT THIS CELL
import numpy as np
import scipy

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
from ipywidgets import interact
from load_data import load_mnist
from numpy.testing import assert_allclose

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

The next cell loads the MNIST digits dataset.

In [ ]:
MNIST = load_mnist('./')
images = MNIST['data'].astype(np.double)
labels = MNIST['target'].astype(np.int)

For this assignment, you need to implement the two functions (`distance` and
`angle`) in the cell below which compute the distance and angle between two
vectors.

### Distances

In [ ]:
# GRADED FUNCTION: DO NOT EDIT THIS LINE
def distance(x0, x1):
    """Compute the distance between two vectors using the dot product.

    Arguments:
        x0: Array of shape (D,).
        x1: Array of shape (D,).

    Returns:
        dist: Distance between x0 and x1.
    """

    # Compute the distance using the dot product:
    dist = np.square(np.subtract(x0, x1))
    dist = np.sqrt(np.sum(dist))

    return dist

In [ ]:
# Test the implementation of 'distance':

# Test case 1:
a = np.array([1, 0])
b = np.array([0, 1])
assert_allclose(distance(a, b), np.sqrt(2), rtol=1e-7)

# Test case 2:
b = np.array([1., np.sqrt(3)])
assert_allclose(distance(a, b), np.sqrt(3), rtol=1e-7)

### Angles

In [ ]:
# GRADED FUNCTION: DO NOT EDIT THIS LINE
def angle(x0, x1):
    """Compute the angle between two vectors using the dot product.

    Arguments:
        x0: Array of shape (D,).
        x1: Array of shape (D,).

    Returns:
        ang: Angle between x0 and x1.
    """

    # Reshape x0 and x1:
    D = len(x0)
    x0_rsh = x0.reshape((D, 1))
    x1_rsh = x1.reshape((D, 1))

    # Compute the dot product of x0 and x1:
    dot_prod = np.matmul(np.transpose(x0_rsh), x1_rsh)
    dot_prod = float(dot_prod)

    # Compute the norm of x0:
    norm_0 = np.sqrt(np.matmul(np.transpose(x0_rsh), x0_rsh))
    norm_0 = float(norm_0)

    # Compute the norm of x1:
    norm_1 = np.sqrt(np.matmul(np.transpose(x1_rsh), x1_rsh))
    norm_1 = float(norm_1)

    # Compute the angle between x0 and x1:
    ang = np.arccos(dot_prod / (norm_0 * norm_1))

    return ang

In [ ]:
# Test the implementation of 'angle':

# Test case 1:
b = np.array([0, 1])
assert_allclose((180 / np.pi) * angle(a, b), 90)

# Test case 2:
b = np.array([1., np.sqrt(3)])
assert_allclose((180 / np.pi) * angle(a, b), 60, rtol=1e-4)

We have created some helper functions for you to visualize vectors in the cells
below. You do not need to modify them.

In [ ]:
def plot_vector(v, w):
    """Plot two 2D vectors."""
    fig = plt.figure(figsize=(4, 4))
    ax = fig.gca()
    plt.xlim([-2, 2])
    plt.ylim([-2, 2])
    plt.grid()
    ax.arrow(
        0, 0, v[0], v[1], head_width=0.05, head_length=0.1,
        length_includes_head=True, linewidth=2, color='r'
    );
    ax.arrow(
        0, 0, w[0], w[1], head_width=0.05, head_length=0.1,
        length_includes_head=True, linewidth=2, color='r'
    );

In [ ]:
b = np.array([0, 1])
plot_vector(a, b)

In [ ]:
# Symmetry tests:
random = np.random.RandomState(42)
for _ in range(10):
    x = random.randn(3)
    y = random.randn(3)
    assert_allclose(distance(x, y), distance(y, x))
    assert_allclose(angle(x, y), angle(y, x), rtol=1e-4)

The next cell shows some digits from the dataset.

In [ ]:
plt.imshow(images[labels==0].reshape(-1, 28, 28)[0], cmap='gray');

But we have the following questions:

1. What does it mean for two digits in the MNIST dataset to be _different_ by
our distance function?
2. Furthermore, how are different classes of digits different for MNIST digits?
Let's find out!

For the first question, we can see just how the distance between digits compare
among all distances for
the first 500 digits. The next cell computes pairwise distances between images.

In [ ]:
distances = [distance(images[i], images[j]) for i in range(500) for j in range(500)]

In [ ]:
@interact(first=(0, 499), second=(0, 499), continuous_update=False)
def show_img(first, second):
    plt.figure(figsize=(8, 4))
    f = images[first].reshape(28, 28)
    s = images[second].reshape(28, 28)
    ax0 = plt.subplot2grid((2, 2), (0, 0))
    ax1 = plt.subplot2grid((2, 2), (1, 0))
    ax2 = plt.subplot2grid((2, 2), (0, 1), rowspan=2)
    ax0.imshow(f, cmap='gray')
    ax1.imshow(s, cmap='gray')
    ax2.hist(np.array(distances), bins=50)
    d = distance(f.ravel(), s.ravel())
    ax2.axvline(x=d, ymin=0, ymax=40000, color='C4', linewidth=4)
    ax2.text(0, 16000, 'Distance is {:.2f}'.format(d), size=12)
    ax2.set(xlabel='Distance', ylabel='Number of images')
    plt.show()

Next we will find the index of the most similar image to the image at index 0.
We will do this by writing some code in another cell.

Write some code in this scratch cell below to find out the most similar image

In [ ]:
# Scratch cell for you to compute the index of the most similar image:
distances = np.array([distance(images[0], images[i]) for i in range(1, 500)])
print(np.argmin(distances) + 1) # Add one since we excluded the 0th image.

Then copy the solution you found (an index value) and replace the -1 in the
function `most_similar_image` with this value. Don't perform any computation in
the next cell that accesses the dataset as the autograder will not have access
to the dataset and will raise an error.

In [ ]:
# GRADED FUNCTION: DO NOT EDIT THIS LINE
def most_similar_image():
    """Find the index of the digit, among all MNIST digits (excluding the
       first), that is the closest to the first image in the dataset."""
    idx = 61
    return idx

For the second question, we can compute a `mean` image for each class of image,
i.e. we compute mean image for digits of `1`, `2`, `3`,..., `9`, then we compute
pairwise distance between them. We can organize the pairwise distances in a 2D
plot, which would allow us to visualize the dissimilarity between images of
different classes.

First we compute the mean for digits of each class.

In [ ]:
mean_images = {}
for n in np.unique(labels):
    mean_images[n] = np.mean(images[labels==n], axis=0)

For each pair of classes, we compute the pairwise distance and
store them into MD (mean distances). We store the angles between the mean digits
in AG

In [ ]:
MD = np.zeros((10, 10))
AG = np.zeros((10, 10))
for i in mean_images.keys():
    for j in mean_images.keys():
        MD[i, j] = distance(mean_images[i], mean_images[j])
        AG[i, j] = angle(mean_images[i].ravel(), mean_images[j].ravel())

Now we can visualize the distances! Here we put the pairwise distances. The
colorbar
shows how the distances map to color intensity.

In [ ]:
fig, ax = plt.subplots()
grid = ax.imshow(MD, interpolation='nearest')
ax.set(
    title='Distances between different classes of digits',
    xlabel='Class of digits',
    xticks=range(10),
    ylabel='Class of digits',
    yticks=range(10)
)
fig.colorbar(grid)
plt.show();

Similarly for the angles.

In [ ]:
fig, ax = plt.subplots()
grid = ax.imshow(AG, interpolation='nearest')
ax.set(
    title='Angles between different classes of digits',
    xlabel='Class of digits',
    xticks=range(10),
    ylabel='Class of digits',
    yticks=range(10)
)
fig.colorbar(grid)
plt.show();